In [ ]:
import glob
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import lmfit as lf

# functions written for this project
import utils

You'll want the mat files and the struct: res.allData. Each row represents a trial. I never added a variable to label the headers so I describe them below. I bolded the variables you probably want to focus on. Hopefully you don't need to feed in the trials in the order they were presented to subjects but if so, you can figure it out with the other variables in the struct.


OS data:

 * Staircase number for a given test block (= file number)
 * Eye (1=weaker eye, 2= fellow eye)
 * Mask Orientation (0=parallel, 90=orthogonal)
 * Binocular condition (1= monocular, 2=dichoptic)
 * Mask Contrast (michelson)
 * Trial number for this staircase
 * Probe contrast recommended by staircase algorithm
 * Response Accuracy (1=correct, 0=incorrect)
 * Probe Contrast used (I don't remember it ever being different from #7 and was really just a sanity check)
 * Interval that probe was presented in (1 or 2)
 * File number (=test block)
 
SS data:

 * Staircase number for a given test block (= file number)
 * Eye (1=weaker eye, 2= fellow eye)
 * Mask Orientation (0=parallel, 90=orthogonal)
 * Binocular Condition (1= monocular, 2=dichoptic)
 * Trial number for this staircase
 * Contrast increment recommended by staircase algorithm
 * Response Accuracy (1=correct, 0=incorrect)
 * Mask Contrast  (michelson)
 * Probe location (1-4, let me know if you need to know which number represents which quadrant)
 * Response (1-4)
 * Probe contrast increment used (I don't remember it ever being different from #6 and was really just a sanity check)
 * File number (=test block)



### Orientation Suppression

In [ ]:
os_data_file = "cleanData/ah_filteredData_OS.mat"
os_data_files = glob.glob("cleanData/*filteredData_OS.mat")

In [ ]:
os_df = utils.load_individual_os_data(os_data_file)

In [ ]:
# sort in order trials were administered, ie block, staricase, trial
os_df.sort_values(["FileNumber", "StaircaseNumber", "TrialNumberStaircase"], inplace=True)


In [ ]:
# Since -1 and -2 represent baseline, these should both be true
print(np.all(os_df[os_df.Orientation==-1]['MaskContrast']==0))
print(np.all(os_df[os_df.Orientation==-2]['MaskContrast']==0))

In [ ]:
os_df.head(n=20)

### Grouping

In [ ]:
gvars = ["Eye", "Orientation", "Presentation"]
gvars_mask = gvars + ["MaskContrast"]
gvars_masktarget = gvars_mask + ["ProbeContrastUsed"]
grouped = os_df.groupby(gvars_masktarget)
for (gv, gr) in grouped:
    print(gv, len(gr))

#### Develop a function that should be minimized to fit the model

Within each staircase, you have numerous trials at different MaskContrasts, and as outcomes you have ResponseAccuracy (0|1). In the paper they describe these as the number of trials (n) and correct responses (c) for each target (here, ProbeContrastUsed) and mask (MaskContrast) level.

Ultimately, they fit their parameters (p, q, z, sigma_int - but mine will be different since i'm using a different model) by minimizing a likelihood function of these parameters given (vectors) m, t, n, c which are the different mask/target conditions.

Questions:
 * what are the parameters of the two-stage model, and which of those are going to be fixed and which are going to be free to vary, ie fit?
 * What is this normal integral d' to percent correct (P(c)) transformation?
     * Phi(t) is the cdf of the normal distribution, tells you the probability that the normal variable is less than t
     * We actually dont use d-prime at all in the two-stage model (they use it for their noise model), so we can just stay in %-correct land
 * What is the proper equation for the likelihood of the parameters given m, t, n, and c?
 * What is the proper function for calculating percent correct from the model parameters?

In [ ]:
params = lf.Parameters()
params.add('w_m', value=1, min=0.0, vary=True)
params.add('w_d', value=0.0249, min=0.0, vary=True)
params.add('a', value=0, vary=False)
params.add('k', value=0.2, vary=False)
params.add('p', value=8, vary=False)
params.add('q', value=6.5, vary=False)
params.add('Z', value=.0084, vary=False)

In [ ]:
s.two_stage_fac_resp(params, [1], [0], [0], [1])

### Surround Suppression

In [ ]:
ss_data_file = "cleanData/ah_filteredData_SS.mat"

In [ ]:
ss_df = utils.load_individual_ss_data(ss_data_file)

In [ ]:
ss_df[ss_df.Orientation==-1]